# Toronto Neighborhood Data

In [3]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df_raw = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df_raw = df_raw[~df_raw['PostalCode'].isnull()]  # to filter out bad rows

In [4]:
df_raw.head(10)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### Delete all rows with Borough values as Not assigned

In [5]:
df_filtered = df_raw.drop(df_raw[df_raw.Borough == 'Not assigned'].index)

In [6]:
df_filtered.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Replace the Neighbourhood names with Borough names for Neighbourhood values showing Not assigned

In [7]:
df_filtered['Neighbourhood'].replace('Not assigned', df_filtered['Borough'], inplace=True)

In [8]:
df_filtered.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Group the rows with same Postal Code to show Neighbourhood in comma separated values

In [9]:
df_grouped = df_filtered.groupby('PostalCode').agg({'Borough':'first',
                                   'Neighbourhood': ','.join}).reset_index()

In [10]:
df_grouped.tail(10)

,PostalCode,Borough,Neighbourhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,Northwest


### Print the shape of the dataframe

In [11]:
df_grouped.shape

(103, 3)

## Update Latitude and Longitude against each Postal Code/ Borough

In [12]:
import pandas as pd

In [13]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data Downloaded')

Data Downloaded


In [14]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_merged = df_grouped

In [16]:
toronto_merged = toronto_merged.join(df_coord.set_index('Postal Code'), on = 'PostalCode')

In [17]:
toronto_merged.tail(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery,Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724


In [18]:
toronto_Neighbourhood = df_filtered
toronto_Neighbourhood = toronto_Neighbourhood.join(df_coord.set_index('Postal Code'), on = 'PostalCode')
toronto_Neighbourhood.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_Neighbourhood['Borough'].unique()),
        toronto_Neighbourhood.shape[0]
    )
)

The dataframe has 11 boroughs and 211 neighborhoods.


## Import necessary libraries for clustering

In [21]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [22]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #################################

In [23]:
toronto_Neighbourhood['Neighbourhood'].count()

211

In [24]:
toronto_nei_grouped = toronto_Neighbourhood.groupby('Borough').count()

In [25]:
toronto_nei_grouped.head()

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,17,17,17,17
Downtown Toronto,37,37,37,37
East Toronto,7,7,7,7
East York,6,6,6,6
Etobicoke,45,45,45,45


In [27]:
toronto_nei_grouped.sort_values(['Neighbourhood'], ascending=False)

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Etobicoke,45,45,45,45
North York,38,38,38,38
Downtown Toronto,37,37,37,37
Scarborough,37,37,37,37
Central Toronto,17,17,17,17
West Toronto,13,13,13,13
York,9,9,9,9
East Toronto,7,7,7,7
East York,6,6,6,6


In [28]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {},{}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963,-79.387207.


In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(toronto_Neighbourhood['Latitude'],toronto_Neighbourhood['Longitude'],toronto_Neighbourhood['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'red',
    fill_color ='blue',
    fill_opacity = 0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

In [37]:
CLIENT_ID = 'IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR' # your Foursquare ID
CLIENT_SECRET = 'G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF' # your Foursquare Secret
VERSION = '20190826' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR
CLIENT_SECRET:G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF


In [40]:
toronto_Neighbourhood.reset_index()
toronto_Neighbourhood.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [126]:
toronto_Neighbourhood = toronto_Neighbourhood[toronto_Neighbourhood['Borough'].str.contains("Toronto")]

In [128]:
toronto_Neighbourhood.reset_index()

,index,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,18,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,19,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,35,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,48,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,49,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,58,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,59,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,69,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


In [129]:
toronto_Neighbourhood.shape

(74, 5)

### Explore the first Neighbourhood in the dataframe

In [132]:
toronto_Neighbourhood.loc[5,'Neighbourhood']

'Harbourfront'

In [134]:
neighbourhood_latitude = toronto_Neighbourhood.loc[5,'Latitude']
neighbourhood_longitude = toronto_Neighbourhood.loc[5,'Longitude']
neighbourhood_name = toronto_Neighbourhood.loc[5,'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [135]:
LIMIT = 100
RADIUS = 500
latitude = neighbourhood_latitude
longitude= neighbourhood_longitude

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, RADIUS, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR&client_secret=G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF&v=20190826&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [136]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6656a95315930039117c01'},
 'response': {'venues': [{'id': '4bc70f5d14d7952126a066e9',
    'name': 'Sackville Playground',
    'location': {'address': '420 king st E',
     'lat': 43.65465604258614,
     'lng': -79.35987064632435,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65465604258614,
       'lng': -79.35987064632435}],
     'distance': 75,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['420 king st E', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566987945',
    'hasPerk': False},
   {'id': '4d067a2b26adb1f702f9dc70',
    'name': 'Thruway Muffler / Downtown Brake',
    'location': {'address'

In [137]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [138]:
import json
from pandas.io.json import json_normalize

In [139]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.postalCode', 'location.state', 'name', 'referralId',
       'venuePage.id'],
      dtype='object')

In [140]:
filtered_columns = ['name','categories','location.lat','location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sackville Playground,Park,43.654656,-79.359871
1,Thruway Muffler / Downtown Brake,Building,43.654242,-79.360853
2,TTC Streetcar #503 Kingston Rd,Moving Target,43.647367,-79.379182
3,Cam's Auto Service,Automotive Shop,43.654195,-79.360545
4,Hastings Barbershop Corktown,Salon / Barbershop,43.654175,-79.361020


In [141]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Explore Neighbourhoods in Toronto

<a id='item2'></a>

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [143]:
toronto_venues = getNearbyVenues(names=toronto_Neighbourhood['Neighbourhood'],
                                   latitudes=toronto_Neighbourhood['Latitude'],
                                   longitudes=toronto_Neighbourhood['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [144]:
print(toronto_venues.shape)
toronto_venues.head()

(3317, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [146]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Berczy Park,57,57,57,57,57,57
Brockton,24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
CN Tower,16,16,16,16,16,16
Cabbagetown,48,48,48,48,48,48
Central Bay Street,84,84,84,84,84,84
Chinatown,100,100,100,100,100,100


In [147]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


<a id='item3'></a>

### Analyze each Neighbourhood

In [148]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
toronto_onehot.shape

(3317, 238)

In [150]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.000000,...,0.00,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Bathurst Quay,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
3,Brockton,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
5,CN Tower,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Cabbagetown,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.020833,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.011905,0.000000,...,0.00,0.000000,0.000000,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
8,Chinatown,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.01,0.000000,0.000000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000
9,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [151]:
toronto_grouped.shape

(73, 238)

In [152]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4        Hotel  0.03


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4     Boat or Ferry  0.06


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2      Farmers Market  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----Brockton----
                   venue  freq
0         Breakfast Spot  0.08
1  Performing Arts Venue  0.08
2            Coffee Shop  0.08
3                   Café  0.08
4                 Bakery  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1  Light Rail Station  0.06
2                Park  0.06
3          Comic Shop  0.06
4    Recording Studio  0.06


----CN Tower----
              venue  freq
0    Airp

In [153]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Burger Joint,Asian Restaurant,Thai Restaurant,American Restaurant,Hotel,Breakfast Spot
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Boat or Ferry,Bar,Plane,Coffee Shop
2,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Bakery,Steakhouse,Italian Restaurant,Seafood Restaurant,Farmers Market,Beer Bar
3,Brockton,Performing Arts Venue,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Intersection,Burrito Place,Caribbean Restaurant,Restaurant,Stadium
4,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Garden Center,Garden,Light Rail Station,Fast Food Restaurant,Farmers Market,Moving Target,Park,Comic Shop


In [155]:
neighborhoods_venues_sorted.shape

(73, 11)

In [156]:
toronto_Neighbourhood.shape

(74, 5)

### Cluster Neighbourhoods

In [157]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 4, 4, 4, 0, 4, 4, 4, 4], dtype=int32)

In [158]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels'] = neighborhoods_venues_sorted['Cluster Labels'].astype(int)

toronto_merged = toronto_Neighbourhood

# merge toronto_grouped with toronto_Neighbourhood to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,4,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Mexican Restaurant,Restaurant,Pub,Theater,Gym / Fitness Center
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Mexican Restaurant,Restaurant,Pub,Theater,Gym / Fitness Center
18,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,4,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Theater,Lingerie Store,Pizza Place,Tea Room,Italian Restaurant
19,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,4,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Theater,Lingerie Store,Pizza Place,Tea Room,Italian Restaurant
35,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Café,Italian Restaurant,Bakery,Hotel,Gastropub,Pizza Place,Breakfast Spot,Cocktail Bar


In [159]:
toronto_merged['Cluster Labels'].unique().tolist()

[4, 1, 2, 3, 0]

In [160]:
toronto_merged.shape

(74, 16)

In [161]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [162]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
158,Central Toronto,2,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
159,Central Toronto,2,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
